In [2]:
from __future__ import print_function
from IPython.core.display import display, HTML


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import plotly.express as px

import matplotlib.ticker as mtick
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import datefinder

import ipywidgets as widgets
from IPython.display import display as ds
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import clear_output
from ipywidgets import Button, Layout

import matplotlib.lines as mlines
import plotly.io as pio
pio.renderers.default='notebook'
import re,datefinder,datetime,pandas as pd,warnings, matplotlib.pyplot as plt
import numpy as np
from ipywidgets import HBox, Label
from matplotlib.pyplot import figure
import collections


warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import random
import time
from dateutil.parser import parse

import geopandas
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from datetime import date, datetime
from shapely.geometry import Point, Polygon

In [3]:
def get_season(date_time):
    # dummy leap year to include leap days(year-02-29) in our range
    leap_year = 2000
    seasons = [('winter', (date(leap_year, 1, 1), date(leap_year, 3, 20))),
               ('spring', (date(leap_year, 3, 21), date(leap_year, 6, 20))),
               ('summer', (date(leap_year, 6, 21), date(leap_year, 9, 22))),
               ('autumn', (date(leap_year, 9, 23), date(leap_year, 12, 20))),
               ('winter', (date(leap_year, 12, 21), date(leap_year, 12, 31)))]

    if isinstance(date_time, datetime):
        date_time = date_time.date()
    # we don't really care about the actual year so replace it with our dummy leap_year
    date_time = date_time.replace(year=leap_year)
    # return season our date falls in.
    return next(season for season, (start, end) in seasons
                if start <= date_time <= end)


def create_season_column(data_set, date_column):
    # cloning the input dataset.
    local = data_set.copy()
    # The apply method calls a function on each row
    local['Season'] = local[date_column].apply(get_season)
    return local

def session(x):
    if 'na' in x:
        return 'No Data'
    x=int(x)
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'

-------------

In [ ]:
ebird=pd.read_csv('./eBirdPreparedDataSet_withObsFreq_v3.0.csv')
ebird=ebird.drop(['SUBSPECIES COMMON NAME', 'SUBSPECIES SCIENTIFIC NAME','GLOBAL UNIQUE IDENTIFIER','GROUP IDENTIFIER','LOCALITY ID','STATE CODE','COUNTY CODE','LOCALITY TYPE','SCIENTIFIC NAME','SAMPLING EVENT IDENTIFIER'], axis = 1)
# ebird['OBSERVATION DATE']=pd.to_datetime(ebird['OBSERVATION DATE'])
ebird['OBSERVATION DATE']=pd.to_datetime(ebird['OBSERVATION DATE'], format='%d-%m-%Y')
ebird['MonthFirstDay']=ebird['OBSERVATION DATE'].values.astype('datetime64[M]')

ebird['Year']=ebird['OBSERVATION DATE'].dt.year
ebird['weekday'] = ebird['OBSERVATION DATE'].apply(lambda x: parse(str(x)).strftime("%A"))
ebird=create_season_column(ebird, date_column='OBSERVATION DATE')
ebird['TOS_H']=[str(a)[:2] for a in ebird['TIME OBSERVATIONS STARTED']]
ebird['session'] = ebird['TOS_H'].apply(session)
ebird['uniqueGEO']=[str(row['LONGITUDE'])+str(row['LATITUDE']) for index, row in ebird.iterrows()]

In [4]:
gdf=geopandas.read_file('./India Districts/IndiaDistricts_2011.shp')

In [ ]:
%%time
geoDistrict={}
for index, row in ebird.drop_duplicates(subset = ["uniqueGEO"]).iterrows():
    s=gdf[gdf.contains(Point( row['LONGITUDE'],row['LATITUDE']))]
    if(len(s)!=0):
        geoDistrict[row['uniqueGEO']]=s.iloc[0]['DISTRICT']
    else:
        geoDistrict[row['uniqueGEO']]='Not Found'
ebird['District']=[geoDistrict[row['uniqueGEO']] for index, row in ebird.iterrows()]
ebird['DistrictFlag']=[row['COUNTY']==row['District'] for index, row in ebird[['COUNTY','District']].iterrows()]

In [ ]:
# ebird.to_csv('ebird_v3.0.csv')

In [10]:
ebird=pd.read_csv('ebird_v3.0.csv', index_col=0)
ebird['OBSERVATION DATE']=pd.to_datetime(ebird['OBSERVATION DATE'], format='%Y-%m-%d')
ebird['MonthFirstDay']=pd.to_datetime(ebird['MonthFirstDay'])

-----

In [ ]:
%%time
ebird_byMonth=pd.DataFrame(columns=['No_Unique_LOCALITY In County','Number of Observations', 'Month','common_name','county', 'EFFORT DISTANCE KM','OBSERVATION COUNT', 'DURATION MINUTES','PROTOCOL_TYPE_Stationary','PROTOCOL_TYPE_Historical','PROTOCOL_TYPE_Traveling','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','TOS_00H','TOS_01H','TOS_02H','TOS_03H','TOS_04H','TOS_05H','TOS_06H','TOS_07H','TOS_08H','TOS_09H','TOS_10H','TOS_11H','TOS_12H','TOS_13H','TOS_14H','TOS_15H','TOS_16H','TOS_17H','TOS_18H','TOS_19H','TOS_20H','TOS_21H','TOS_22H','TOS_23H','TOS_naH','session_Morning','session_Noon','session_Eve','session_Early_Morning','session__No_Data','session_Late_Night','session_Night','Season_summer','Season_autumn','Season_spring','Season_winter'])
count=0
for county, row in ebird.sort_values('OBSERVATION DATE').groupby('COUNTY'):
    for common_name, row1 in row.groupby('COMMON NAME'):
        for index, row2 in row1.groupby('Year'):
            for index, row3 in row2.sort_values('OBSERVATION DATE').groupby('MonthFirstDay'):
                ebird_byMonth.loc[count] = [row3['LOCALITY'].unique().size,len(row3), index.date(),common_name,county, int(sum(row3['EFFORT DISTANCE KM'])), int(np.nansum(row3['OBSERVATION COUNT'])), int(np.nansum(row3['DURATION MINUTES'])),len(row3[row3['PROTOCOL TYPE']=='Stationary']),len(row3[row3['PROTOCOL TYPE']=='Historical']),len(row3[row3['PROTOCOL TYPE']=='Traveling']),len(row3[row3['weekday']=='Monday']),len(row3[row3['weekday']=='Tuesday']),len(row3[row3['weekday']=='Wednesday']),len(row3[row3['weekday']=='Thursday']),len(row3[row3['weekday']=='Friday']),len(row3[row3['weekday']=='Saturday']),len(row3[row3['weekday']=='Sunday']),    len(row3[row3['TOS_H']=='00']),len(row3[row3['TOS_H']=='01']),len(row3[row3['TOS_H']=='02']),len(row3[row3['TOS_H']=='03']),len(row3[row3['TOS_H']=='04']),len(row3[row3['TOS_H']=='05']),len(row3[row3['TOS_H']=='06']),len(row3[row3['TOS_H']=='07']),len(row3[row3['TOS_H']=='08']),len(row3[row3['TOS_H']=='09']),len(row3[row3['TOS_H']=='10']),len(row3[row3['TOS_H']=='11']),len(row3[row3['TOS_H']=='12']),len(row3[row3['TOS_H']=='13']),len(row3[row3['TOS_H']=='14']),len(row3[row3['TOS_H']=='15']),len(row3[row3['TOS_H']=='16']),len(row3[row3['TOS_H']=='17']),len(row3[row3['TOS_H']=='18']),len(row3[row3['TOS_H']=='19']),len(row3[row3['TOS_H']=='20']),len(row3[row3['TOS_H']=='21']),len(row3[row3['TOS_H']=='22']),len(row3[row3['TOS_H']=='23']),len(row3[row3['TOS_H']=='na']),len(row3[row3['session']=='Morning']), len(row3[row3['session']=='Noon']), len(row3[row3['session']=='Eve']), len(row3[row3['session']=='Early Morning']), len(row3[row3['session']=='No Data']), len(row3[row3['session']=='Late Night']), len(row3[row3['session']=='Night']),len(row3[row3['Season']=='summer']),len(row3[row3['Season']=='autumn']),len(row3[row3['Season']=='spring']),len(row3[row3['Season']=='winter'])]
                count=count+1

In [ ]:
LastBirdObservarionByMonth=[]
for index, row in ebird_byMonth.groupby(['county','common_name']):
    flagCount=0
    flagDate=''
    for index, row1 in row.iterrows():
        if flagCount==0:
            LastBirdObservarionByMonth.append(0)
            flagCount=flagCount+1
            flagDate=row1['Month']
        else:
            flagCount=flagCount+1
            LastBirdObservarionByMonth.append((row1['Month']-flagDate).days)
            flagDate=row1['Month']
ebird_byMonth['LastBirdObservarionByMonth']=LastBirdObservarionByMonth
ebird_byMonth['LastBirdObservarionByDays']=LastBirdObservarionByMonth
ebird_byMonth['LastBirdObservarionByMonth']=round(ebird_byMonth['LastBirdObservarionByMonth']/30,2)

In [ ]:
FoundInTWOSession=[]
FoundInONESession=[]
FoundMoreThanTWOSession=[]
for index , row in ebird_byMonth.iterrows():
    flagCount=0
    if row['session_Morning']!=0:
        flagCount=flagCount+1
    if row['session_Eve']!=0:
          flagCount=flagCount+1
    if row['session_Early_Morning']!=0:
          flagCount=flagCount+1
    if row['session_Late_Night']!=0:
          flagCount=flagCount+1
    if row['session_Late_Night']!=0:
          flagCount=flagCount+1
    if row['session_Night']!=0:
          flagCount=flagCount+1
    if flagCount==1:
        FoundInONESession.append(True)
    else:
        FoundInONESession.append(False)
    if flagCount==2:
        FoundInTWOSession.append(True)
    else:
        FoundInTWOSession.append(False)
    if flagCount>2:
        FoundMoreThanTWOSession.append(True)
    else:
        FoundMoreThanTWOSession.append(False)
ebird_byMonth['FoundInONESession']=FoundInONESession
ebird_byMonth['FoundInTWOSession']=FoundInTWOSession
ebird_byMonth['FoundMoreThanTWOSession']=FoundMoreThanTWOSession

In [ ]:
# ebird_byMonth.to_csv('ebird_byMonth.csv')

In [5]:
ebird_byMonth=pd.read_csv('ebird_byMonth.csv', index_col=0)
ebird_byMonth['Month']=pd.to_datetime(ebird_byMonth['Month'], format='%Y-%m-%d')

In [ ]:
%%time
county_brid_found_count= pd.DataFrame(columns=['County','Common Name','Number of Unique Localities in County','Number of Observations','Duration in Days','Start Date', 'End Date'])
count=0
for index, row in ebird.groupby('COUNTY'):
    for index1, row1 in row.groupby('COMMON NAME'):
        county_brid_found_count.loc[count]=[index,index1,row['LOCALITY'].unique().size,len(row1),(list(row.sort_values('OBSERVATION DATE')['OBSERVATION DATE'])[-1]-list(row.sort_values('OBSERVATION DATE')['OBSERVATION DATE'])[0]).days,list(row.sort_values('OBSERVATION DATE')['OBSERVATION DATE'])[0],list(row.sort_values('OBSERVATION DATE')['OBSERVATION DATE'])[-1]]
        count=count+1
county_brid_found_count.sort_values(['County','Number of Observations'],inplace=True)

In [ ]:
# county_brid_found_count.to_csv('county_brid_found_count.csv')

In [6]:
county_brid_found_count=pd.read_csv('county_brid_found_count.csv', index_col=0)

In [ ]:
%%time
county_brid_by_month_found_count= pd.DataFrame(columns=['County','Month','Common Name','Number of Unique Localities in County','Number of Observations','Duration in Days','Start Date', 'End Date'])
count=0
for index2, row2 in ebird.groupby('MonthFirstDay'):
    for index, row in row2.groupby('COUNTY'):
        for index1, row1 in row.groupby('COMMON NAME'):
            county_brid_by_month_found_count.loc[count]=[index,index2,index1,row['LOCALITY'].unique().size,len(row1),(list(row.sort_values('OBSERVATION DATE')['OBSERVATION DATE'])[-1]-list(row.sort_values('OBSERVATION DATE')['OBSERVATION DATE'])[0]).days,list(row.sort_values('OBSERVATION DATE')['OBSERVATION DATE'])[0],list(row.sort_values('OBSERVATION DATE')['OBSERVATION DATE'])[-1]]
            count=count+1
county_brid_by_month_found_count.sort_values(['County','Number of Observations'],inplace=True)

In [ ]:
# county_brid_by_month_found_count.to_csv('county_brid_by_month_found_count.csv')

In [7]:
county_brid_by_month_found_count=pd.read_csv('county_brid_by_month_found_count.csv', index_col=0)

In [8]:
gdf.head(2)

,DISTRICT,ST_NM,geometry
0,Adilabad,Telangana,"POLYGON ((78.84972 19.76010, 78.85102 19.75945..."
1,Agra,Uttar Pradesh,"POLYGON ((78.19803 27.40280, 78.19804 27.40278..."


In [11]:
ebird.head(2)

,COMMON NAME,STATE,OBSERVATION_FREQUENCY,LOCALITY,LATITUDE,LONGITUDE,TIME OBSERVATIONS STARTED,DURATION MINUTES,EFFORT DISTANCE KM,COUNTY,...,OBSERVATION COUNT,MonthFirstDay,Year,weekday,Season,TOS_H,session,uniqueGEO,District,DistrictFlag
0,Purple Sunbird,Karnataka,1.0,East of Raya Gopura,12.657685,76.649596,10:09:00,15,0.0,Mandya,...,2,2015-03-01,2015,Sunday,winter,10,Morning,76.649595612.6576845,Mandya,True
1,Southern Hill Myna,Tamil Nadu,1.0,Valparai Plateau,10.329072,76.953735,13:05:00,25,10.0,Coimbatore,...,1,2015-06-01,2015,Thursday,spring,13,Noon,76.953735410.3290717,Coimbatore,True


In [12]:
ebird_byMonth.head(2)

,No_Unique_LOCALITY In County,Number of Observations,Month,common_name,county,EFFORT DISTANCE KM,OBSERVATION COUNT,DURATION MINUTES,PROTOCOL_TYPE_Stationary,PROTOCOL_TYPE_Historical,...,session_Night,Season_summer,Season_autumn,Season_spring,Season_winter,LastBirdObservarionByMonth,LastBirdObservarionByDays,FoundInONESession,FoundInTWOSession,FoundMoreThanTWOSession
0,2,5,2017-01-01,Ashy Drongo,Adilabad,150,25,0,0,5,...,0,0,0,0,5,0.0,0,False,False,False
1,1,1,2019-03-01,Ashy Drongo,Adilabad,2,1,120,0,0,...,0,0,0,0,1,26.3,789,True,False,False


In [13]:
county_brid_found_count.head(2)

,County,Common Name,Number of Unique Localities in County,Number of Observations,Duration in Days,Start Date,End Date
12,Adilabad,Tree Pipit,27,3,1578,2015-11-05,2020-03-01
5,Adilabad,Booted Eagle,27,4,1578,2015-11-05,2020-03-01


In [14]:
county_brid_by_month_found_count.sort_values(['Month','County','Common Name'],inplace=True)

In [15]:
county_brid_by_month_found_count

,County,Month,Common Name,Number of Unique Localities in County,Number of Observations,Duration in Days,Start Date,End Date
0,Alappuzha,2015-01-01,Bay-backed Shrike,17,1,27,2015-01-04,2015-01-31
1,Alappuzha,2015-01-01,Baya Weaver,17,10,27,2015-01-04,2015-01-31
2,Alappuzha,2015-01-01,Black Drongo,17,44,27,2015-01-04,2015-01-31
3,Alappuzha,2015-01-01,Blyth's Reed Warbler,17,14,27,2015-01-04,2015-01-31
4,Alappuzha,2015-01-01,Brown Shrike,17,5,27,2015-01-04,2015-01-31
...,...,...,...,...,...,...,...,...
32799,Yanam,2020-10-01,Baya Weaver,3,3,0,2020-10-17,2020-10-17
32800,Yanam,2020-10-01,Black Drongo,3,6,0,2020-10-17,2020-10-17
32801,Yanam,2020-10-01,Brown Shrike,3,3,0,2020-10-17,2020-10-17
32802,Yanam,2020-10-01,Greater Coucal,3,2,0,2020-10-17,2020-10-17


In [16]:
from pycaret.anomaly import *
from pycaret.datasets import get_data
anomaly = county_brid_by_month_found_count
exp_ano = setup(anomaly)

,Description,Value
0,session_id,279
1,Original Data,"(32804, 8)"
2,Missing Values,False
3,Numeric Features,3
4,Categorical Features,2
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(32804, 190)"
9,CPU Jobs,-1


#### iforest

In [17]:
%%time
iforest_model_county_brid_by_month_found_count=create_model('iforest')
save_model(iforest_model_county_brid_by_month_found_count, 'iforest_model_county_brid_by_month_found_count')

Transformation Pipeline and Model Succesfully Saved
Wall time: 9.59 s


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='UNSUPERVISED_DUMMY_TARGET',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='most frequent',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None...
                 ('fix_perfect', 'passthrough'),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  IFo

In [18]:
iforest_model_county_brid_by_month_found_count = load_model('iforest_model_county_brid_by_month_found_count')

Transformation Pipeline and Model Successfully Loaded


In [19]:
%%time
# plot_model(iforest_model_county_brid_by_month_found_count)

Wall time: 0 ns


In [20]:
Outlier_results_iforest =assign_model(iforest_model_county_brid_by_month_found_count)

In [21]:
len(Outlier_results_iforest[Outlier_results_iforest['Anomaly']==1])

1641

In [26]:
Outlier_results_iforest[Outlier_results_iforest['Anomaly']==1]

,County,Month,Common Name,Number of Unique Localities in County,Number of Observations,Duration in Days,Start Date,End Date,Anomaly,Anomaly_Score
156,Kozhikode,2015-01-01,Ashy Drongo,1,1,0,2015-01-29,2015-01-29,1,0.002300
159,Krishnagiri,2015-01-01,Ashy Drongo,6,1,3,2015-01-15,2015-01-18,1,0.001512
407,Alappuzha,2015-02-01,Baya Weaver,200,16,22,2015-02-05,2015-02-27,1,0.005767
409,Alappuzha,2015-02-01,Blyth's Reed Warbler,200,37,22,2015-02-05,2015-02-27,1,0.001093
410,Alappuzha,2015-02-01,Booted Eagle,200,1,22,2015-02-05,2015-02-27,1,0.001075
...,...,...,...,...,...,...,...,...,...,...
32591,Salem,2020-10-01,Ashy Drongo,104,12,30,2020-10-01,2020-10-31,1,0.003906
32600,Salem,2020-10-01,Indian Peafowl,104,78,30,2020-10-01,2020-10-31,1,0.004330
32714,Tumkur,2020-10-01,Ashy Drongo,18,1,29,2020-10-02,2020-10-31,1,0.001553
32724,Udupi,2020-10-01,Ashy Drongo,58,28,30,2020-10-01,2020-10-31,1,0.001437


In [23]:
len(Outlier_results_iforest)

32804

In [28]:
import plotly.express as px
fig = px.line(Outlier_results_iforest[(Outlier_results_iforest['Anomaly']==1)&(Outlier_results_iforest['County']=='Kozhikode')], x='Month', y="Number of Observations")
fig.show()

In [41]:
df = Outlier_results_iforest[Outlier_results_iforest['Anomaly']==1]
fig = px.line(df, x="Month", y='Number of Observations')
# fig.update_xaxes(
#     dtick="Month",
#     tickformat="%Month")
fig.show()

#### KNN

In [42]:
%%time
knn_model_county_brid_by_month_found_count=create_model('knn')
save_model(knn_model_county_brid_by_month_found_count, 'knn_model_county_brid_by_month_found_count')

Transformation Pipeline and Model Succesfully Saved
Wall time: 11.4 s


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='UNSUPERVISED_DUMMY_TARGET',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='most frequent',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None...
                 ('fix_perfect', 'passthrough'),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  KNN

In [43]:
knn_model_county_brid_by_month_found_count = load_model('knn_model_county_brid_by_month_found_count')

Transformation Pipeline and Model Successfully Loaded


In [ ]:
Outlier_results_knn =assign_model(knn_model_county_brid_by_month_found_count)

In [ ]:
len(Outlier_results_knn[Outlier_results_knn['Anomaly']==1])

In [ ]:
len(Outlier_results_knn)

In [ ]:
all_models = models()

In [ ]:
all_models